In [7]:
import chess.pgn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import lichess.api
from lichess.format import PGN

pgn = open("lichess_db_standard_rated_2016-05.pgn")

users = set()
dfGames = pd.DataFrame(columns = ['Event', 'White', 'Black', 'Opening', 'WhiteElo', 'BlackElo'])
for x in range(100):
    game = chess.pgn.read_game(pgn)
    if game.headers.get('Event') == "Rated Classical game":
        row = [game.headers.get('Event'), game.headers.get('White'), game.headers.get('Black'), game.headers.get('Opening'), game.headers.get('WhiteElo'), game.headers.get('BlackElo')]
        dfGames = dfGames.append(pd.Series(row, index = ['Event', 'White', 'Black', 'Opening', 'WhiteElo', 'BlackElo']), ignore_index = True)
        users.add(game.headers.get('White'))
        users.add(game.headers.get('Black'))

In [18]:
from lichess.format import PYCHESS
#print(dfGames['White'])
print(users)
pgns = lichess.api.user_games('JOHN14-6', max=50, format=PYCHESS)
print(next(pgns))

{'pepi_1800', 'Sahinsah', 'aerdna87', 'mynlan', 'Biterolf', 'mariofil', 'JOHN14-6', 'Muthos', 'dreypat', 'yonyo30', 'Newton_10', 'TenAndOnly10', 'caporipens', 'valerioneto', 'madri1enians', 'nader10', 'Sacchina', 'filobertito', 'raking41', 'zoranastanisic', 'moe1337', 'diskborste', 'Shankh', 'naikus', 'Spiedo', 'brunoctba32', 'chuecheng', 'Gambit47', 'platito_de_lefa', 'el_sheikh', 'a7md2010', 'PaMaViGi', 'edumenoyo', 'Dexter_Ni', 'guevara', 'PiqueFain'}
[Event "Rated Rapid game"]
[Site "https://lichess.org/7s905hVw"]
[Date "2022.10.23"]
[Round "?"]
[White "JOHN14-6"]
[Black "mirnyi60"]
[Result "0-1"]
[BlackElo "1496"]
[BlackRatingDiff "+6"]
[ECO "D00"]
[Termination "Normal"]
[TimeControl "600+5"]
[UTCDate "2022.10.23"]
[UTCTime "22:51:44"]
[Variant "Standard"]
[WhiteElo "1477"]
[WhiteRatingDiff "-11"]

1. d4 d5 2. g3 Bf5 3. Bg2 e6 4. c3 Nf6 5. Nf3 h6 6. Qb3 b6 7. Qb5+ Qd7 8. Qb3 Nc6 9. Ne5 Qd6 10. Nxc6 Qxc6 11. Nd2 Bd6 12. O-O O-O 13. Qd1 Qb7 14. Nf3 c5 15. Nh4 Bh7 16. Qe1 c4 17. e4 d

In [11]:
basketMap = {}
for user in users:
    try:
        pgns = lichess.api.user_games(user, max=20, format=PYCHESS)
    except:
        continue
    openings = set()
    for pgn in pgns:
        if pgn.headers.get('Event') == "Rated Classical game":
            print(pgn.headers.get('Opening'))
            openings.add(pgn.headers.get('Opening'))
    basketMap[user] = list(openings)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [10]:
print(basketMap)

{'pepi_1800': [], 'Sahinsah': [None], 'mynlan': [], 'Biterolf': [None], 'mariofil': [], 'JOHN14-6': [], 'Muthos': [], 'dreypat': [], 'Newton_10': [], 'TenAndOnly10': [], 'caporipens': [], 'valerioneto': [], 'madri1enians': [None], 'nader10': [], 'Sacchina': [None], 'filobertito': [], 'raking41': [None], 'zoranastanisic': [], 'moe1337': [], 'diskborste': [], 'Shankh': [], 'naikus': [], 'Spiedo': [None], 'brunoctba32': [], 'chuecheng': [], 'Gambit47': [None], 'platito_de_lefa': [], 'a7md2010': [], 'PaMaViGi': [None], 'edumenoyo': [], 'Dexter_Ni': [None], 'guevara': [None], 'PiqueFain': []}


In [14]:
print(dfGames)

                   Event           White            Black  \
0   Rated Classical game       edumenoyo  platito_de_lefa   
1   Rated Classical game        a7md2010          dreypat   
2   Rated Classical game        raking41         mariofil   
3   Rated Classical game        JOHN14-6           naikus   
4   Rated Classical game         guevara         Sacchina   
5   Rated Classical game        PaMaViGi        pepi_1800   
6   Rated Classical game       PiqueFain      brunoctba32   
7   Rated Classical game       chuecheng        el_sheikh   
8   Rated Classical game        Sahinsah         aerdna87   
9   Rated Classical game         yonyo30         Biterolf   
10  Rated Classical game          mynlan       caporipens   
11  Rated Classical game          Shankh          moe1337   
12  Rated Classical game      diskborste      filobertito   
13  Rated Classical game  zoranastanisic      valerioneto   
14  Rated Classical game    madri1enians           Muthos   
15  Rated Classical game